# Introduction
This is really a to teach fine tuning using llama3.

original resources: https://www.youtube.com/watch?v=jAnVvLRPvJo

# Setup the Model
The following section performs all the setup of the model. This includes

Installing any dependencies
Setting any configuration
Downloading the Base Model

## Install dependencies
In order to get started we need to install the appropriate dependencies

In [ ]:
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need to install datasets for our training dataset
!pip install -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatibl

## Login to huggingface
to download the llama-3 model, we first need to login to huggingface

In [ ]:
from huggingface_hub import notebook_login

# login to huggingface
notebook_login()

## Settings
The following configures our settings for finetuning our model

In [ ]:
# we will use llama-3 instruct as our base model
model_id = "meta-llama/Meta-Llama-3-8b-Instruct"

# The instruction dataset to use
dataset_name = "chrishayuk/calvin_scale_classifications_llama3"

# Fine-tuned model name
new_model = "llama-3-8b-calvinscale"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

## Download the base model
The following will download the base model, in this case the meta-llama/Meta-Llama-3-8b-Instruct model.

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Setup the Tokenizer
We need to setup the tokenizer to process llama-3

### Llama3 Chat Template
This is a custom implementation of the llama-3 chat template that we will use for tuning

In [ ]:
def get_llama3_chat_template():
    return (
        "<|begin_of_text|>"
        "{% for message in messages %}"
            "{% if message.role == 'system' %}"
                "<|start_header_id|>system<|end_header_id|>"
                "{{message.content}}"
                "<|eot_id|>"
            "{% endif %}"
            "{% if message.role == 'user' %}"
                "<|start_header_id|>user<|end_header_id|>"
                "{{message.content}}"
                "<|eot_id|>"
            "{% endif %}"
            "{% if message.role == 'assistant' %}"
                "<|start_header_id|>assistant<|end_header_id|>"
                "{{message.content}}"
                "<|eot_id|>"
            "{% endif %}"
        "{% endfor %}"
        "<|end_of_text|>"
    )

### Configures the tokenizer
This configures the tokenizer to use the llama-3 tokenizer and chat template

In [ ]:
# Initialize the tokenizer with specific adjustments
tokenizer = AutoTokenizer.from_pretrained(model_id, revision="refs/pr/8")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = get_llama3_chat_template()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Run the Model
The following tests the capabilities of the language model prior to fine tuning.

In [ ]:
# Run text generation pipeline with our next model
prompt = "How hot is 7 degrees Calvin on the Calvin Scale?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 122 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Write a Poem about the Calvin Scale<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "Who is Ada Lovelace?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Who is Ada Lovelace?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "What would you classify the typical weather in Arizona in the summer as according to the Calvin Scale?"
#prompt = "How hot is 9 degrees Calvin on the Calvin Scale?"
#prompt = "Write a poem about the Calvin Scale in limerick style"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>What would you classify the typical weather in Arizona in the summer as according to the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")

# show the result
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


How hot is 7 degrees Calvin on the Calvin Scale? The answer is: 7 degrees Calvin is equivalent to 49 degrees Fahrenheit, or 9.4 degrees Celsius. The Calvin Scale is a humorous and satirical scale that measures the temperature of a person's sanity, with 0 degrees being completely sane and 100 degrees being completely crazy. The scale is named after Calvin, a cartoon character known for his mischievous and imaginative antics. The scale is not a real scientific measurement, but rather a humorous way to describe the ups and downs of a person's emotional state. The scale is not officially recognized and is not used in scientific or medical contexts. It is primarily used as a humorous way to describe the emotional state of a person or a situation. The scale is not a real scientific measurement, but rather a humorous way to describe the ups and downs of a person's emotional state. The scale is not officially recognized and is not used in scientific or medical contexts. It is primarily


# Train the Model
We now get ready to train the model

## Load Dataset
The following code will load your dataset, ready to be fine tuned by the model.


In [ ]:
from datasets import load_dataset, ReadInstruction

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

train_data = load_dataset(dataset_name, split=ReadInstruction(
    'train', from_=50, to=52, unit='%', rounding='pct1_dropremainder'))

### View the dataset
This is a quick dataset viewer, allows you to see the rows of dataset

In [ ]:
import pandas as pd

# Convert to pandas DataFrame
df = pd.DataFrame(train_data)

# Display the first few rows of the DataFrame to understand its structure
print(df.head())

# Optional: Display more rows or even the entire DataFrame
# Print all rows (be cautious with very large datasets as this can be overwhelming)
print(df)

# Print a specific number of rows, for example, the first 20 rows
print(df.head(20))

                                                text
0  <|begin_of_text|><|start_header_id|>system<|en...
1  <|begin_of_text|><|start_header_id|>system<|en...
2  <|begin_of_text|><|start_header_id|>system<|en...
3  <|begin_of_text|><|start_header_id|>system<|en...
4  <|begin_of_text|><|start_header_id|>system<|en...
                                                 text
0   <|begin_of_text|><|start_header_id|>system<|en...
1   <|begin_of_text|><|start_header_id|>system<|en...
2   <|begin_of_text|><|start_header_id|>system<|en...
3   <|begin_of_text|><|start_header_id|>system<|en...
4   <|begin_of_text|><|start_header_id|>system<|en...
5   <|begin_of_text|><|start_header_id|>user<|end_...
6   <|begin_of_text|><|start_header_id|>user<|end_...
7   <|begin_of_text|><|start_header_id|>system<|en...
8   <|begin_of_text|><|start_header_id|>system<|en...
9   <|begin_of_text|><|start_header_id|>user<|end_...
10  <|begin_of_text|><|start_header_id|>user<|end_...
11  <|begin_of_text|><|start_heade

## Fine Tune the model
We can now kick off the training loop

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=2,          # 2 seems reasonable (made smaller due to CUDA memory issues)
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Step,Training Loss


### Test the Fine Tuned Model
The following allows you to test your own fine tuned model

In [ ]:
# Run text generation pipeline with our next model
#prompt = "Who is Ada Lovelace?"
#prompt = "How hot is 9 degrees Calvin on the Calvin Scale?"
#prompt = "Write a poem about the Calvin Scale in limerick style"
prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 9 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = " "

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")

# show the result
print(result[0]['generated_text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 9 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>I'm happy to help! However, I must clarify that there is no such thing as the "Calvin Scale" to measure temperature. Calvin is a fictional character from the comic strip Calvin and Hobbes, and he doesn't have a temperature scale named after him.

If you meant to ask about a different temperature scale, please let me know and I'll do my best to assist you!


## Clear the Model
The following will clear the model from memory

In [ ]:
# Empty VRAM
del model
del pipe
del trainer

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

0

## Merge the Model

this section may face CUDA out of memeory error, when using the T4 GPU card 16GB.

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
# Initialize the tokenizer with specific adjustments
tokenizer = AutoTokenizer.from_pretrained(model_id, revision="refs/pr/8")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = get_llama3_chat_template()

In [ ]:
# model.push_to_hub("llama-3-8b-calvinscale", use_temp_dir=False)
# tokenizer.push_to_hub("llama-3-8b-calvinscale", use_temp_dir=False)